In [2]:
# 1 Setup & Imports
%load_ext autoreload
%autoreload 2

import sys
import os
import gc
import joblib
import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt

# Setup path
current_dir = Path.cwd()
root_dir = current_dir.parent
if str(root_dir) not in sys.path:
    sys.path.append(str(root_dir))

from src import config, preprocessing, autoencoder, rf_classifier, evaluation, utils
from sklearn.model_selection import train_test_split

# Khởi tạo folder experiment
exp_paths = utils.setup_experiment_folder()
print(f"🚀 Experiment initialized at: {exp_paths['root']}")

# 2: Load Data
YEAR_TARGET = "2018" # ĐỔI THÀNH "2018" KHI CHẠY FILE 2018

try:
    print(f"🚀 Loading {YEAR_TARGET} dataset...")
    X, y = preprocessing.load_single_dataset_year(YEAR_TARGET, binary_mode=True)
    print(f"✅ Loaded {YEAR_TARGET}: {X.shape}")
except Exception as e:
    print(f"❌ Lỗi khi nạp dữ liệu: {e}")

# 3: Split Data
print(f"✂️ Splitting {YEAR_TARGET} (80/20)...")

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=config.SEED, stratify=y
)

print(f"📊 Train Size: {X_train.shape}")
print(f"📊 Test Size:  {X_test.shape}")

# Giải phóng RAM dữ liệu gốc
del X
gc.collect()

# 4: Scale Data
print("🚀 Scaling data...")
scaler = preprocessing.get_scaler()

# Chỉ fit trên tập train của năm hiện tại
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Lưu scaler
scaler_path = exp_paths["models"] / f"scaler_{YEAR_TARGET}.joblib"
joblib.dump(scaler, scaler_path)
print(f"✅ Saved scaler to: {scaler_path}")

# 5: Train AE
L = 30  # Input dimension
N = 5   # Latent dimension

print(f"🚀 Training AE on {YEAR_TARGET} (L={L}, N={N})...")

# Cắt dữ liệu lấy 30 đặc trưng đầu tiên cho AE
X_train_ae_in = X_train_scaled[:, :L]

ae_model = autoencoder.DeepAutoencoder(
    input_dim=L,
    latent_dim=N,
    hidden_layers=[22, 12]
)

ae_save_path = exp_paths["models"] / f"ae_model_{YEAR_TARGET}.pth"
ae_model = autoencoder.train_ae(ae_model, X_train_ae_in, save_path=ae_save_path)

print("✅ AE training done.")

# 6 
print("🧠 Extracting Latent Features...")

# Trích xuất Z (N=5) cho cả Train và Test
X_train_latent = autoencoder.extract_features(ae_model, X_train_scaled[:, :L])
X_test_latent  = autoencoder.extract_features(ae_model, X_test_scaled[:, :L])

print(f"✅ Latent features shape: {X_train_latent.shape}")

# 7 : 
print("🔗 Fusing features...")

# Lấy index của 20 đặc trưng mRMR (đã định nghĩa trong config.py)
all_feature_names = config.SELECTED_FEATURES
mrmr_indices = [all_feature_names.index(feat) for feat in config.mRMR_FEATURES]

# Thực hiện Fusion
X_train_fusion = np.hstack([X_train_scaled[:, mrmr_indices], X_train_latent])
X_test_fusion  = np.hstack([X_test_scaled[:, mrmr_indices], X_test_latent])

print(f"🔥 Fusion Completed. Final Shape: {X_train_fusion.shape}")

# 8 
print(f"🚀 Training RF on {YEAR_TARGET} Fusion Data...")

rf_save_path = exp_paths["models"] / f"rf_model_{YEAR_TARGET}.joblib"
rf_model = rf_classifier.train_rf(X_train_fusion, y_train, save_path=rf_save_path)

print("\n--- FINAL EVALUATION ---")
# Hàm này sẽ tự động tính Accuracy, F1, Recall, MCC và VẼ LUÔN AUROC
evaluation.evaluate_model(
    model=rf_model,
    X_test=X_test_fusion,
    y_test=y_test,
    save_dir=exp_paths["figures"],
    dataset_name=f"Within_{YEAR_TARGET}"
)



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
📂 New Experiment Created: C:\Users\Admin\Documents\ids_ae_rf_hybrid\results\experiments\exp_20251221_063426
🚀 Experiment initialized at: C:\Users\Admin\Documents\ids_ae_rf_hybrid\results\experiments\exp_20251221_063426
🚀 Loading 2018 dataset...
🔄 Loading dataset year 2018 (Binary=True)...
✅ Loaded 2018. Shape: (9625148, 65)
✅ Loaded 2018: (9625148, 65)
✂️ Splitting 2018 (80/20)...
📊 Train Size: (7700118, 65)
📊 Test Size:  (1925030, 65)
🚀 Scaling data...
✅ Saved scaler to: C:\Users\Admin\Documents\ids_ae_rf_hybrid\results\experiments\exp_20251221_063426\models\scaler_2018.joblib
🚀 Training AE on 2018 (L=30, N=5)...
[Autoencoder] Training on cuda...
  Epoch 5/50 - Loss: 0.227854
  Epoch 10/50 - Loss: 0.122332
  Epoch 15/50 - Loss: 0.092471
  Epoch 20/50 - Loss: 0.083993
  Epoch 25/50 - Loss: 0.079946
  Epoch 30/50 - Loss: 0.077630
  Epoch 35/50 - Loss: 0.076131
  Epoch 40/50 - Loss: 0.074242
  Epoch 4

[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.


building tree 1 of 100building tree 2 of 100

building tree 3 of 100
building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100


[Parallel(n_jobs=8)]: Done  25 tasks      | elapsed:  2.2min


building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
building tree 43 of 100
building tree 44 of 100
building tree 45 of 100
building tree 46 of 100
building tree 47 of 100
building tree 48 of 100
building tree 49 of 100
building tree 50 of 100
building tree 51 of 100
building tree 52 of 100
building tree 53 of 100
building tree 54 of 100
building tree 55 of 100
building tree 56 of 100
building tree 57 of 100
building tree 58 of 100
building tree 59 of 100
building tree 60 of 100
building tree 61 of 100
building tree 62 of 100
building tree 63 of 100
building tree 64 of 100
building tree 65 of 100
building tree 66 of 100
building tree 67 of 100
building tree 68 of 100
building tree 69 of 100
building tree 70 of 100
building tree 71 of 100
building tree 72 of 100
building tree 73 of 100
building tree 74 of 100
building tree 75

[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:  7.2min finished


✅ RF Model saved to: C:\Users\Admin\Documents\ids_ae_rf_hybrid\results\experiments\exp_20251221_063426\models\rf_model_2018.joblib

--- FINAL EVALUATION ---

📊 Evaluating on Within_2018...


[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  25 tasks      | elapsed:    0.6s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    2.0s finished


   ✅ Accuracy: 0.9781
   ⭐ MCC:      0.9461
   📝 Report saved to: report_Within_2018.txt
   🖼️ Confusion Matrix saved to: cm_Within_2018.png


{'accuracy': 0.9781120294229181,
 'mcc': 0.9461326341037732,
 'report': {'Normal': {'precision': 0.9810932678234814,
   'recall': 0.9884126821401747,
   'f1-score': 0.9847393741531684,
   'support': 1375383.0},
  'Attack': {'precision': 0.970453441505712,
   'recall': 0.9523366815428812,
   'f1-score': 0.9613097123778618,
   'support': 549647.0},
  'accuracy': 0.9781120294229181,
  'macro avg': {'precision': 0.9757733546645967,
   'recall': 0.970374681841528,
   'f1-score': 0.9730245432655151,
   'support': 1925030.0},
  'weighted avg': {'precision': 0.9780553158871048,
   'recall': 0.9781120294229181,
   'f1-score': 0.9780495857832147,
   'support': 1925030.0}}}